# CreditCardApproval MachineLearning Diana Max

## Setup notebook & fetch data

In [ ]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
import random

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, TargetEncoder, label_binarize
from sklearn.metrics import (
    accuracy_score,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix,
    classification_report,
    mean_squared_error,
    make_scorer,
    RocCurveDisplay,
    PrecisionRecallDisplay,
    DetCurveDisplay
)

from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV, 
    KFold, 
    RandomizedSearchCV, 
    train_test_split, 
    StratifiedKFold, 
    cross_validate,
    RepeatedStratifiedKFold
)

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import clone
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

In [ ]:
credit_approval = fetch_ucirepo(id=27)

X = credit_approval.data.features
y = credit_approval.data.targets
df = credit_approval.data.original

In [ ]:
y

## preprocessing

In [ ]:
"""
Maybe add some more advanced techniques from here later: https://scikit-learn.org/stable/modules/preprocessing.html

"""

### feature engineering: mixed numerical categorical feature space (this is old code, use fully numerical feature space for now)

In [ ]:
make_column_transformer(
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(drop='first')
        ),
        ['A1']
        
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='median'),
            MinMaxScaler()
        ),
        ['A2']
    ),
    (
        make_pipeline(
            FunctionTransformer(lambda col: col.mask(col != 'u', 'non-u')),
            SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(drop='first')
           ),
        ['A4']
    ),
    (
        make_pipeline(
            FunctionTransformer(lambda col: col.replace({'v':'v', 'h':'h','bb':'other','ff':'other','j':'other','z':'other','dd':'other','n':'other','o':'other'})),
            SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(drop='first')
        ),
        ['A7']
    ),
    (
        make_pipeline(
            FunctionTransformer(lambda col: col.replace({'g':'g', 's':'non-g','p':'non-g'})),
            SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(drop='first')
        ),
        ['A13']
    ),
    (
        'test',
        make_pipeline(
            FunctionTransformer(np.log1p),
            SimpleImputer(strategy='median'),
            MinMaxScaler()
        ),
        ['A11','A14','A15']
    ),
    # continuous default
    (
        make_pipeline(
            MinMaxScaler(),
        ),
        ['A3','A8']
    ),
    # categorical default
    (
        make_pipeline(
            OneHotEncoder(drop='first'),
        ),
        ['A9','A10', 'A12']
    ),
    
    # remove: A5, A6
    remainder='drop'
)

### feature engineering: fully numerical feature space

#### variant 1: include A7

In [ ]:
column_tweaker_include_a7 = make_column_transformer(
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(sparse_output=False, drop='first')
        ),
        ['A1','A9','A10','A12']
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            FunctionTransformer(lambda col: label_binarize(col, classes=['u']))
        ),
        ['A4']
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            FunctionTransformer(lambda col: label_binarize(col, classes=['g']))
        ),
        ['A5']
    ),
    (
        make_pipeline(
            TargetEncoder(),
            SimpleImputer(strategy='median')
        ),
        ['A6', 'A7']
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            FunctionTransformer(lambda col: label_binarize(col, classes=['g']))
        ),
        ['A13']
    ),
    (
        make_pipeline(
            FunctionTransformer(np.log1p),
            SimpleImputer(strategy='median')
        ),
        ['A2','A3','A8','A11','A14','A15']
    ),

    # leave everything else untouched
    remainder='drop'
)

#### variant 2: exclude A7

In [ ]:
column_tweaker_exclude_a7 = make_column_transformer(
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(sparse_output=False, drop='first')
        ),
        ['A1','A9','A10','A12']
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            FunctionTransformer(lambda col: label_binarize(col, classes=['u']))
        ),
        ['A4']
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            FunctionTransformer(lambda col: label_binarize(col, classes=['g']))
        ),
        ['A5']
    ),
    (
        make_pipeline(
            TargetEncoder(),
            SimpleImputer(strategy='median')
        ),
        ['A6']
    ),
    (
        make_pipeline(
            SimpleImputer(strategy='most_frequent'),
            FunctionTransformer(lambda col: label_binarize(col, classes=['g']))
        ),
        ['A13']
    ),
    (
        make_pipeline(
            FunctionTransformer(np.log1p),
            SimpleImputer(strategy='median')
        ),
        ['A2','A3','A8','A11','A14','A15']
    ),

    # leave everything else untouched
    remainder='drop'
)

### feature scaling

In [ ]:
column_scaler = MinMaxScaler()

### PCA

In [ ]:
column_pca = PCA(n_components=.99, svd_solver='full')

### put together pipelines

#### feature engineering pipeline 1

In [ ]:
preprocessing_pipe = make_pipeline(
    column_tweaker_exclude_a7,
    column_scaler,
    column_pca
)
preprocessing_pipe

#### feature engineering pipeline 2

In [ ]:
"""

In which different formats do we need the data for different classifiers ?

"""

## train test validation split

In [ ]:
# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)

In [ ]:
X_preprocessed = preprocessing_pipe.fit_transform(X=X, y=y.values.ravel())

In [ ]:
X_train, X_test_validate, y_train, y_test_validate = train_test_split(X_preprocessed, y.values.ravel(), test_size=0.1, random_state=seed)
X_train.shape, y_train.shape

In [ ]:
X_test, X_validate, y_test, y_validate = train_test_split(X_test_validate, y_test_validate, test_size=.5)
(X_test.shape, y_test.shape), (X_validate.shape, y_validate.shape)

In [ ]:
X_train_test, y_train_test = np.concatenate((X_train, X_test)), np.concatenate((y_train, y_test)) 

In [ ]:
np.concatenate((X_train))

## train & optimise different models

On what to optimize ?

The task is o predict, if, given the different parameters, access to a credit card is denied or granted.

- The worst case would be to give access to a position that is obvious fraud (False Positive). Cost: The owner looses money.
- Rejecting access to a legit position (False Negative) would be inconvenient, since someone wouldn't be able to draw his money, but we consider it less of a problem than a False Positive.

- Obviously, a True Positive would be giving someone his money while a True Negative would be denying fraud.



In summary, we primarily optimize for the False Positive rate.

Additional metrics:
- The dataset is balanced therefore classificatino accuracy is a meaningfull metrix as well.

How to compare classifiers ?
- Plot learning curves.
- Compare missclassified examples.
- Compare robustness in different CV-splits.

Next, we compare the performance of a variety of common ML classifiers. Since we are relativey naive to which classifiers could work, we try a diverse set.

### Discriminant Analysis

#### Linear Discriminant Analysis

In [ ]:
y.value_counts()

In [ ]:
lda = LinearDiscriminantAnalysis()

param_grid = {
    'priors':[[.3,.7],[.4,.6],[.5,.5],[.6,.4],[.7,.3]]
    }

In [ ]:

lda_gs = GridSearchCV(estimator=lda, param_grid=param_grid, cv=10)

lda_gs.fit(X_train_test, y_train_test)

lda_gs.best_estimator_.score(X_validate, y_validate)

#### Quadratic Discriminant Analysis

In [ ]:
qda = QuadraticDiscriminantAnalysis()

param_grid = {
    'priors':[[.3,.7],[.4,.6],[.5,.5],[.6,.4],[.7,.3]]
    }

qda_gs = GridSearchCV(estimator=qda, param_grid=param_grid, cv=10)

qda_gs.fit(X_train_test, y_train_test)

qda_gs.best_estimator_.score(X_validate, y_validate)

### Gaussian Process Classifier

In [ ]:
gpc = GaussianProcessClassifier()

In [ ]:
gpc_cv_result = cross_val_score(gpc, X_preprocessed, y.values.ravel(), cv=10)

In [ ]:
gpc_cv_result.mean()

### Random Forest

In [ ]:
clf = RandomForestClassifier(max_depth=6, random_state=seed)

clf.fit(X_train, y_train)

In [ ]:
clf.score(X_validate, y_validate)

#### GridSearchCV

In [ ]:
param_grid = [{'n_estimators': [50, 100, 200], 'min_samples_split': [3, 5], 'min_samples_leaf': [3, 6]}]

# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_depth': [None, 10, 20, 30, 40, 50],
#     'min_samples_split': [2, 5, 10, 15, 20],
#     'min_samples_leaf': [1, 2, 4, 6, 8, 10],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'bootstrap': [True, False],
#     'criterion': ['gini', 'entropy']
# }

In [ ]:
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

In [ ]:
rf = RandomForestClassifier(random_state=seed)

In [ ]:
cv = StratifiedKFold(n_splits=10)

In [ ]:
search_rf = GridSearchCV(
    estimator=rf, scoring=scoring, param_grid=param_grid, 
    verbose=1, cv=cv, refit='AUC')

In [ ]:
search_rf.fit(X_train_test, y_train_test)

In [ ]:
search_rf.cv_results_

In [ ]:
search_rf.best_params_ , search_rf.best_estimator_

In [ ]:
# Auskommentiert, weil search_rf.best_estimator_ eigentlich genau das macht
# rf_best = RandomForestClassifier(
#     n_estimators=search_rf.best_params_['n_estimators'], 
#     min_samples_split=search_rf.best_params_['min_samples_split'],
#     min_samples_leaf=search_rf.best_params_['min_samples_leaf'])

In [ ]:
## mit den besten Parametern fitten
rf_best = search_rf.best_estimator_.fit(X_train_test, y_train_test)
accuracy_score(y_validate, search_rf.best_estimator_.predict(X_validate))

### KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

In [ ]:
knn.score(X_validate, y_validate)

#### GridSearch

In [ ]:
param_grid = [{'n_neighbors':[5, 10, 15], 'weights':['uniform', 'distance'], 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], 'p':[1, 2]}]


In [ ]:
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

In [ ]:
neighbor = KNeighborsClassifier()

In [ ]:
cv = StratifiedKFold(n_splits=10)

In [ ]:
search_knn = GridSearchCV(
    estimator=neighbor, scoring=scoring, param_grid=param_grid, 
    verbose=1, cv=cv, refit='AUC')

In [ ]:
search_knn.fit(X_train_test, y_train_test)

In [ ]:
search_knn.cv_results_

In [ ]:
search_knn.best_params_ , search_knn.best_estimator_

In [ ]:
# Auskommentiert, weil search_knn.best_estimator_ eigentlich genau das macht
# knn_best = KNeighborsClassifier(
#     algorithm=search_knn.best_params_['algorithm'], 
#     n_neighbors=search_knn.best_params_['n_neighbors'],
#     p=search_knn.best_params_['p'],
#     weights=search_knn.best_params_['weights'])

In [ ]:
## mit den besten Parametern fitten
search_knn.best_estimator_.fit(X_train_test, y_train_test)
accuracy_score(y_validate, search_knn.best_estimator_.predict(X_validate))

### Ensemble method

### Adaboost

In [ ]:
ada = AdaBoostClassifier(LinearDiscriminantAnalysis())

In [ ]:
ada_cv_result = cross_val_score(ada, X_preprocessed, y.values.ravel(), cv=10)

In [ ]:
"""
Check if all these methods always categorize the same, or different samples wrong. If the later is the case, this would motivate using a ensemble method to balance strengths and weaknesses of different classifiers.

Maybe building some ensemble method that combines a (gradient boosted) dewcicion tree for the categorical variables with something like SVM for the continuous data.

This would obviate the need for target encoding of A6 and A7 and poosible better handle the many binary variables in the dataset. 
""";

## (Sequential Feature Selection)

In [ ]:
"""
Further reduce dimensionality by SFS. This may give different results for different classifiers.

Compare results of SFS to results of EDA
"""

## model comparison

In [ ]:

print('Best RandomForestClassifier Accuracy:', accuracy_score(y_validate, search_rf.best_estimator_.predict(X_validate)))
print('Best KNeighborsClassifier Accuracy:', accuracy_score(y_validate, search_knn.best_estimator_.predict(X_validate)))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 4))

RocCurveDisplay.from_estimator(search_rf.best_estimator_, X_preprocessed, y, ax=ax1)
RocCurveDisplay.from_estimator(search_knn.best_estimator_, X_preprocessed, y, ax=ax1)
ax1.set_title("ROC curve")

PrecisionRecallDisplay.from_estimator(search_rf.best_estimator_, X_preprocessed, y, ax=ax2)
PrecisionRecallDisplay.from_estimator(search_knn.best_estimator_, X_preprocessed, y, ax=ax2)
ax2.set_title("Precision-Recall curve")



In [ ]:
y_num = np.where(y_train == '+', 1, 0)
y_validate_num = np.where(y_validate == '+', 1, 0)

rf_num = search_rf.best_estimator_.fit(X_train, y_num)
knn_num = search_knn.best_estimator_.fit(X_train, y_num)




In [ ]:
#DetCurveDisplay.from_predictions(y_validate, rf_best.predict)
# Get prediction probabilities
# probabilities = rf_best.predict_proba(X_validate)
# print("Prediction Probabilities:\n", probabilities)

# Get class predictions (optional)
predictions_rf = rf_num.predict(X_validate)
print("Class Predictions:\n", predictions_rf)

predictions_knn = knn_num.predict(X_validate)
print("Class Predictions:\n", predictions_knn)

DetCurveDisplay.from_predictions(y_validate_num, predictions_rf)
DetCurveDisplay.from_predictions(y_validate_num, predictions_knn)

In [ ]:
# Model evauation

# Since classes are balanced, accuracy is the correct evaluation metric.

# Plot ROC for different hyperparameters

# above: plot full results of grid search to showcase effects of different hyperparameters